### Preprocess the data / train-test split into separate files / store the embeddings into npy array for later use

#### Make imports

In [2]:
import os 
import json
import numpy as np
import re
import keras
import pickle
import spacy
import threading
from time import time
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import random
import gensim
import pickle
from copy import deepcopy

In [3]:
os.chdir('disk/ERIK_ML/')

#### Load text data

In [24]:
X = []
titles = []
y = []
for file_name in os.listdir('DataSet/'):
    if not file_name.endswith('.json'):
        continue
    else:
        file = open(os.path.join('DataSet/',file_name),'r')
        data = json.load(file)
        # extract questions
        for line in data:
            X.append(line['question'])
            y.append(file_name[:-5])
            titles.append(line['title'])

In [19]:
X[1]

'Hallo,\n1) es existiert ein neu gegründetes Gewerbe (Einzelunternehmer). Es soll nun ein Fahrzeug geleast werden und komplett abgesetzt werden (sämtliche Kosten, Leasingrate, Leasingsonderzahlung, Benzin, Vorsteuer, usw.) Die Bank kann das Leasingangebot nur privat ausstellen, gewerblich geht dies nicht, da für das neu gegründete Gewerbe keine Zahlen vorliegen. Kann ein auf die Privatperson abgeschlossener Leasingvertrag trotzdem vollständig ins Gewerbe eingebracht werden, um alles abzusetzen? Im Leasingvertrag steht ausdrücklich „Privatleasing". Oder geht dies nur mit einem gewerblichen Leasingvertrag?\n2) Falls möglich - gäbe es hier ein Risiko im Falle einer Betriebsprüfung, da der Leasingvertrag als „Privatleasing" deklariert ist, sodass die Betriebsausgaben nicht anerkannt werden o. Ä.?\nVielen Dank.'

In [25]:
titles[1]

'Privates Leasing im Gewerbe?'

In [5]:
titles_output = deepcopy(titles)

#### Preprocess texts

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;.\\n]')
BAD_SYMBOLS_RE = re.compile('[^A-Za-züäößÖÄÜ ]')

PHRASES=re.compile(r"\bgute?n? tag\b|\bsehr geehrte damen und herren\b|\bha?e?llo\b|\bviele? grü?u?s?s?ß?e?\b|\bgute?n? morgen\b|\bdanke\b|\bbitten?\b|\bgar\b",
                   re.IGNORECASE)

In [7]:
f = open('stopwords.pkl','rb')
STOPWORDS = pickle.load(f)

In [8]:
STOPWORDS

['im',
 'und',
 'diese',
 'sind',
 'ich',
 'kommen',
 'habe',
 'damit',
 'leben',
 'man',
 'möglich',
 'euro',
 'das',
 'unter',
 'mein',
 'sein',
 'mutter',
 'wenn',
 'jahr',
 'jahren',
 'dies',
 'mal',
 'zu',
 'bekommen',
 'oder',
 'auf',
 'für',
 'hier',
 'ohne',
 'neu',
 'so',
 'noch',
 'sagen',
 'vater',
 'war',
 'kein',
 'welch',
 'monat',
 'mich',
 'mehr',
 'hatte',
 'seit',
 'dass',
 'meine',
 'soll',
 'die',
 'ist',
 'ein',
 'lassen',
 'er',
 'geben',
 'vom',
 'den',
 'haben',
 'wie',
 'bereits',
 'zahlen',
 'um',
 'viel',
 'verkaufen',
 'es',
 'einer',
 'auch',
 'nur',
 'in',
 'hausen',
 'machen',
 'folgend',
 'da',
 'eine',
 'deutschland',
 'wegen',
 '1',
 'dieser',
 'wohnung',
 'nicht',
 'meiner',
 'sich',
 'aus',
 'einen',
 'wird',
 'nichts',
 'sehen',
 'vor',
 'muss',
 'von',
 'zeit',
 'ja',
 'bei',
 'ca',
 'bin',
 'der',
 'also',
 'werden',
 'hat',
 'jahre',
 'recht',
 'durch',
 'über',
 'bis',
 'als',
 'uns',
 'dann',
 'mit',
 'immer',
 'vertragen',
 'schreiben',
 'dürf

In [9]:
STOPWORDS_REMOVE = re.compile(r'\b(?:%s)\b' % '|'.join(STOPWORDS),re.IGNORECASE)

In [10]:
nlp = spacy.load('de_core_news_sm')

In [11]:
def text_prepare(text):
    
    """
        text: a string
        
        return: modified initial string
    """
    # lowercase text
    #text = text.lower()
    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    # delete symbols which are in BAD_SYMBOLS_RE from text
    text = BAD_SYMBOLS_RE.sub('',text)
    text = text.lower()
    text = PHRASES.sub('',text)
    # delete stopwords from text
    text = STOPWORDS_REMOVE.sub('',text)
    
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text) # delete words that are occuring multiple times in a row
    text =  re.sub(r"\b[a-zA-ZÖÄÜöäüß]\b", "", text) # delete single lettre words
    text = re.sub(' +', ' ', text) # delete multiple spaces
    
    return text
    

In [12]:
def titles_prepare(text,start):
    
    """
        text: a string
        
        return: modified initial string
    """
    # lowercase text
    #text = text.lower()
    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(r"([?.!,¿-])", r" \1 ", text) # keep the punctuation
    text = re.sub(r'[" "]+', " ", text)
    #text = text.lower()
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text) # delete words that are occuring multiple times in a row
    text =  re.sub(r"\b[a-zA-ZÖÄÜöäüß]\b", "", text) # delete single lettre words
    text = re.sub(' +', ' ', text) # delete multiple spaces
    if start:
        return '<start> ' + text.lower() # for teacher forcing
    else:
        return text.lower() + ' <stop>'
    

In [13]:
# preprocess in-place
def process_text(text_array,n):
    for i in range(len(text_array)):
        X[n*10000+i] = text_prepare(text_array[i])
    return

In [14]:
# preprocess in-place
def process_titles(text_array,n):
    text_array_ = deepcopy(text_array)
    for i in range(len(text_array)):
        titles[n*10000+i] = titles_prepare(text_array_[i],True)
        titles_output[n*10000+i] = titles_prepare(text_array_[i],False)
    return

In [15]:
# do it in parallel stop programming like the first man on earth... 
thread_list = []
start = time()
for i in range(10):
    thread_list.append(threading.Thread(target = process_text,args = ((X[i*10000:i*10000+10000]),i)))
    thread_list[i].start()
for thread in thread_list:
    thread.join()
stop = time()
print('preprocessing X took : ', stop-start,' seconds')

preprocessing X took :  364.50031185150146  seconds


In [17]:
X[1]

' existiert gegründetes gewerbe einzelunternehmer fahrzeug geleast komplett abgesetzt sämtliche leasingrate leasingsonderzahlung benzin vorsteuer usw bank leasingangebot privat ausstellen gewerblich gegründete gewerbe vorliegen privatperson abgeschlossener leasingvertrag vollständig gewerbe eingebracht abzusetzen leasingvertrag ausdrücklich privatleasing gewerblichen leasingvertrag falls gäbe risiko falle betriebsprüfung leasingvertrag privatleasing deklariert sodass betriebsausgaben anerkannt '

In [21]:
# do it in parallel stop programming like the first man on earth... 
thread_list = []
start = time()
for i in range(10):
    thread_list.append(threading.Thread(target = process_titles,args = ((titles[i*10000:i*10000+10000]),i)))
    thread_list[i].start()
for thread in thread_list:
    thread.join()
stop = time()
print('preprocessing titles took : ', stop-start,' seconds')

preprocessing titles took :  4.610769748687744  seconds


In [22]:
titles[1]

'<start> privates leasing im gewerbe ? '

#### Prepare texts for use in a deep learning algorithm

In [16]:
# shuffle titles, lables and questions
dataset = list(zip(X,titles,titles_output,y))
random.shuffle(dataset)
mixed_X, mixed_titles,mixed_titles_output, mixed_y= zip(*dataset)

In [17]:
print('question: ',mixed_X[1])
print('\ntitle: '+mixed_titles[1])
print('\ntitle output: '+mixed_titles_output[1])
print('\narea: ',mixed_y[1])

question:   geburt deutscher staatsbürger ehefrau mexikannerin ehefrau befindet zweimonatigen intensiven deutschkurs offiziellen goethe instituts mexico intensiven deutschkurs verlangt familienleben führen absolvieren job ingenieurin gekündigt fahrtzeit eingerechnet täglich stunden erlernen einfachen deutschkenntnisse angemessene krankenversicherung hochzeit defekt gallenblase festgestellt operative entfernung gallenblase jederzeit meist tödlich verlaufender gallenblasenriss entstehen ärztliche diagnose ausreichenden finanziellen operation mexikanischen krankenhaus westlichem niveau arbeitet staatlichen krankenhäuser mexico befinden schrecklichen hygienischen fachlichen technischen zustand notwendig schnell aufenthaltserlaubnis bekommt krankenversicherung aufnehmen operiert deutschprüfung dauer testergebnis deutschprüfung feststeht monate vergehen verlängerte visabearbeitungszeit monaten aufgrund flüchtlingskrise studierte ingenieurin leider dauert monate offiziellen abschluss privatun

In [18]:
MAX_LEN_Q = len(max(X, key=lambda x: len(x.split())).split())
print('maximal length of a question {} words'.format(MAX_LEN_Q))

maximal length of a question 2840 words


In [19]:
MAX_LEN_T = len(max(titles, key=lambda x: len(x.split())).split())
print('maximal length of a title {} words'.format(MAX_LEN_T))

maximal length of a title 29 words


In [20]:
train_X = mixed_X[:99000]
train_y = mixed_y[:99000]
train_titles_input = mixed_titles[:99000]
train_titles_output = mixed_titles_output[:99000]

In [21]:
test_X = mixed_X[99000:]
test_y = mixed_y[99000:]
test_titles_input = mixed_titles[99000:]
test_titles_output = mixed_titles_output[99000:]

In [22]:
print('question: ',test_X[1])
print('\ntitle: '+test_titles_input[1])
print('\ntitle output: '+test_titles_output[1])
print('\narea: ',test_y[1])

question:   liebe gemeinde folgenden vorfall erlitten meinung mithilfe monaten anfang september gebrauchtwagen skoda octavia freien händler zugelegt letzte urlaub fahrbetrieb kfz fällt elektronik bedeutet auto gefahren fahrtrichtungsanzeiger etc funktionierte auto demnach verkehrssicher km heimatort entfernt rücksprache versicherung adac nächste fachwerkstatt gleichen ort geleitet bordsteuergerät glaube hieß verdacht getauscht keinerlei fehler festzustellen symptomatik beschriebenen probleme hinwies kostenpunkt hause blieb autobahn selben problem liegen nächstgelegenen fachwerkstatt geschleppt anderthalb mitgeteilt bordsteuergerät ausfall schuld gateway diagnoseinterface wasser kfz läuft betroffenen teile laufe korrodierten lasse überprüfen wasser auto läuft werkstatt sowieso halbe konsole auseinander gebaut kostenpunkt entschuldigt elend langen text anliegen verkäufer wagens gewährleistung agbs geregelt zudem klausel informieren fragen werdegang reparatur leider gedanken agbs kopf aut

In [23]:
tokenizer_inputs = Tokenizer(num_words=150000,filters = '') #filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
tokenizer_inputs.fit_on_texts(train_X)
train_sequences = tokenizer_inputs.texts_to_sequences(train_X)
train_sequences = pad_sequences(train_sequences,maxlen=700)
test_sequences = tokenizer_inputs.texts_to_sequences(test_X)
test_sequences = pad_sequences(test_sequences,maxlen=700)
# get the word to index mapping for input language
word2idx_inputs = tokenizer_inputs.word_index
print('Found %s unique input tokens.' % len(word2idx_inputs))

Found 299331 unique input tokens.


In [24]:
idx2word_X = {}
for key, value in word2idx_inputs.items():
    idx2word_X[value]=key

In [25]:
f = open('word2idx_X_generator.pkl','wb')
pickle.dump(word2idx_inputs,f)

In [26]:
f = open('idx2word_X_generator.pkl','wb')
pickle.dump(idx2word_X,f)

In [27]:
train_sequences

array([[    0,     0,     0, ...,   200,  1015,  2276],
       [    0,     0,     0, ...,  1443, 12160, 15680],
       [    0,     0,     0, ...,   577,  1094,   223],
       ...,
       [    0,     0,     0, ...,  2335,   747,  1352],
       [    0,     0,     0, ...,   557,   303,  1044],
       [    0,     0,     0, ...,   555,   387,   165]], dtype=int32)

In [28]:
np.save('X_train_padded_generator.npy',np.array(train_sequences))

In [29]:
np.save('X_test_padded_generator.npy',np.array(test_sequences))

In [30]:
tokenizer_inputs = Tokenizer(num_words=30000) #filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
tokenizer_inputs.fit_on_texts(train_titles_input+train_titles_output)

input_sequences_input = tokenizer_inputs.texts_to_sequences(train_titles_input)
input_sequences_input = pad_sequences(input_sequences_input,maxlen=20,padding = 'post')
input_sequences_output = tokenizer_inputs.texts_to_sequences(train_titles_output)
input_sequences_output = pad_sequences(input_sequences_output,maxlen=20,padding = 'post')

test_sequences_input =  tokenizer_inputs.texts_to_sequences(test_titles_input)
test_sequences_input = pad_sequences(test_sequences_input,maxlen=20, padding = 'post')
test_sequences_output =  tokenizer_inputs.texts_to_sequences(test_titles_output)
test_sequences_output = pad_sequences(test_sequences_output,maxlen=20, padding = 'post')
# get the word to index mapping for input language
word2idx_inputs = tokenizer_inputs.word_index
print('Found %s unique input tokens.' % len(word2idx_inputs))

Found 47623 unique input tokens.


In [31]:
input_sequences_input[0]

array([    1,  6115,   369, 14192,    46,  1890,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0], dtype=int32)

In [33]:
input_sequences_output[0]

array([ 6115,   369, 14192,    46,  1890,     2,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0], dtype=int32)

In [34]:
idx2word_titles = {}
for key, value in word2idx_inputs.items():
    idx2word_titles[value]=key

#### Save preprocessed data to files

In [35]:
f = open('word2idx_titles_generator.pkl','wb')
pickle.dump(word2idx_inputs,f)

In [36]:
f = open('idx2word_titles_generator.pkl','wb')
pickle.dump(idx2word_titles,f)

In [37]:
np.save('titles_train_input_padded_generator.npy',np.array(input_sequences_input))

In [38]:
np.save('titles_train_output_padded_generator.npy',np.array(input_sequences_output))

In [39]:
np.save('titles_test_input_padded_generator.npy',np.array(test_sequences_input))

In [40]:
np.save('titles_test_output_padded_generator.npy',np.array(test_sequences_output))

In [41]:
np.save('y_train_generator.npy',np.array(train_y))

In [42]:
np.save('y_test_generator.npy',np.array(test_y))

#### Save word2vec as numpy array

In [43]:
# store all the pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
with open(os.path.join('trained_vectorsW2V.csv')) as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
    for line in f:
        values = line.split(',')
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Loading word vectors...
Found 151456 word vectors.


In [44]:
f = open('word2idx_X_generator.pkl','rb')
word2idx_X = pickle.load(f)

In [45]:
num_words = 150000
EMBEDDING_DIM = 50

In [46]:
# prepare embedding matrix
print('Filling pre-trained embeddings...')
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx_X.items():
    if i < num_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all zeros.
          embedding_matrix[i] = embedding_vector

Filling pre-trained embeddings...


In [47]:
embedding_matrix.shape

(150000, 50)

In [48]:
np.save('embeddings_generator.npy',embedding_matrix)